# Enhancing RAG with Contextual Retrieval

We will use an LLM to generate for each chunk and document a contextual sentence to improve its retrival accuracy and use in hybrid search.

* Generate the context sentence.
* Enrich the chunk embedding vectors with the context.
* Create a topic database to be used in hybrid search.
* Perform hybrid search to improve retrieval results.

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich) to make the output more readable, and supress warning messages.

In [2]:
from rich.pretty import pprint
from rich.theme import Theme
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

custom_theme = Theme({
    "repr.own": "bright_yellow",            # Class names
    "repr.tag_name": "bright_yellow",       # Adjust tag names which might still be purple
    "repr.call": "bright_yellow",           # Function calls and other symbols
    "repr.str": "bright_green",             # String representation
    "repr.number": "bright_red",            # Numbers
    "repr.attrib_name": "bright_yellow",    # Attribute names
    "repr.attrib_value": "bright_blue"      # Attribute values
})

# Apply the theme and print the object with rich formatting

console = Console(theme=custom_theme)

In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading a complex dataset of documents

We will load a complex dataset of scientific documents from Arxiv. Applying naive chunks on such documents will give poor results in RAG applications.

In [4]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv2", split="train")
console.print(dataset)

Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 
'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [5]:
import os
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

encoder = OpenAIEncoder(name="text-embedding-3-small")

In [6]:
from semantic_chunkers import StatisticalChunker
import logging

logging.disable(logging.CRITICAL)

chunker = StatisticalChunker(
    encoder=encoder,
    min_split_tokens=100,
    max_split_tokens=500,
)

In [7]:
chunks_0 = chunker(docs=[dataset["content"][0]])


In [11]:
first_chunk = ' '.join(chunks_0[0][0].splits)
console.print(first_chunk)

4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. Jiang, Alexandre 
Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las 
Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio Renard Lavaud, 
Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le 
Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed Abstract We introduce 
Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same architecture as Mistral 7B, 
with the difference that each layer is composed of 8 feedforward blocks (i.e. experts). For every token, at each 
layer, a router network selects two experts to process the current state and combine their outputs. Even though 
each token only sees two experts, the selected experts can be different at each timestep. As a result, each token 
has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was trained with a 
context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. 
In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and multilingual benchmarks.
We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that surpasses GPT-3.5 Turbo, 
Claude-2.1, Gemini Pro, and Llama 2 70B â

In [18]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
import anthropic

client = anthropic.Anthropic(
    # This is the default and can be omitted
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)


In [20]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""

def situate_context(doc: str, chunk: str) -> str:
    response = client.beta.prompt_caching.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.0,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc),
                        "cache_control": {"type": "ephemeral"} #we will make use of prompt caching for the full documents
                    },
                    {
                        "type": "text",
                        "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk),
                    }
                ]
            }
        ],
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    return response

In [23]:
chunk_context = situate_context(dataset["content"][0], first_chunk)

In [24]:
console.print(chunk_context)

PromptCachingBetaMessage(
    id='msg_01U87paNMV7Q5odMtg4LpWD8',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on various benchmarks. It also describes the model architecture and the fine-tuned Mixtral 
8x7B - Instruct model.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=12532,
        input_tokens=584,
        output_tokens=73
    )
)

In [25]:
second_chunk = ' '.join(chunks_0[0][1].splits)

In [26]:
second_chunk_context = situate_context(dataset["content"][0], second_chunk)

In [27]:
console.print(second_chunk_context)

PromptCachingBetaMessage(
    id='msg_01WgBrxDRVLdrPoPCTbWsiyt',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural details of Mixtral, including its 
use of a sparse mixture-of-experts network, and mentions that the base and instruct models are released under the 
Apache 2.0 license.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=12532,
        input_tokens=290,
        output_tokens=92
    )
)

In [42]:
arxiv_id = dataset[0]["id"]
refs = list(dataset[0]["references"].values())
doc_text = dataset[0]["content"]
title = dataset[0]["title"]

corpus_json = []
for i, chunk in enumerate(chunks_0[0]):
    chunk_text = ' '.join(chunk.splits)
    contextualized_text = situate_context(doc_text, chunk_text).content[0].text
    corpus_json.append({
        "text": f"{chunk_text}\n\n{contextualized_text}",
        "metadata" : {
            "title": title,
            "arxiv_id": arxiv_id,
            "references": refs
        }
    })

In [43]:
console.print(corpus_json[:2])

[
    {
        'text': '4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. 
Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio 
Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon 
Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed 
Abstract We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same 
architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward blocks (i.e. experts).
For every token, at each layer, a router network selects two experts to process the current state and combine their
outputs. Even though each token only sees two experts, the selected experts can be different at each timestep. As a
result, each token has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was 
trained with a context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all 
evaluated benchmarks. In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and 
multilingual benchmarks. We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that 
surpasses GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B â\n\nThis chunk introduces Mixtral 8x7B, a sparse 
mixture of experts language model that outperforms Llama 2 70B and GPT-3.5 on various benchmarks. It also describes
the model architecture and the fine-tuned Mixtral 8x7B - Instruct model.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    {
        'text': 'chat model on human bench- marks. Both the base and instruct models are released under the Apache 
2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/mixtral-of-experts/ # 
Introduction In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights, 
licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As it only uses a subset
of its parameters for every token, Mixtral allows faster inference speed at low batch-sizes, and higher throughput 
at large batch-sizes. Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the 
feedforward block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router 
network chooses two of these groups (the â\n\nThis chunk introduces Mixtral 8x7B, a sparse mixture of experts 
language model that outperforms Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural 
details of Mixtral, including its use of a sparse mixture-of-experts network, and mentions that the base and 
instruct models are released under the Apache 2.0 license.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    }
]

## Hybrid Search

We will use bm25 supported database to complement the semantic search with the vector database.

In [35]:
import bm25s

In [44]:
corpus_text = [doc["text"] for doc in corpus_json]

# Tokenize the corpus and only keep the ids (faster and saves memory)
corpus_tokens = bm25s.tokenize(corpus_text, stopwords="en")

# Create the BM25 retriever and attach your corpus_json to it
retriever = bm25s.BM25(corpus=corpus_json)
# Now, index the corpus_tokens (the corpus_json is not used yet)
retriever.index(corpus_tokens)




Split strings:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/46 [00:00<?, ?it/s]

In [45]:
# Query the corpus
query = "What is context size of Mixtral?"
query_tokens = bm25s.tokenize(query)


results, scores = retriever.retrieve(query_tokens, k=2)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    console.print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 2.11): {'text': "Active Params French Arc-c HellaS MMLU German Arc-c HellaS MMLU Spanish Arc-c 
HellaS MMLU Italian Arc-c HellaS MMLU 33B 70B 13B 42.9% 65.4% 49.0% 39.3% 68.1% 49.9% 49.9% 72.5% 64.3% 49.4% 70.9%
65.1% 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9% 41.1% 63.3% 48.7% 47.3% 68.7% 64.2% 
45.7% 69.8% 52.3% 50.5% 74.5% 66.0% Table 4: Comparison of Mixtral with Llama on Multilingual Benchmarks. On ARC 
Challenge, Hellaswag, and MMLU, Mixtral outperforms Llama 2 70B on 4 languages: French, German, Spanish, and 
Italian. # 3.2 Long range performance To assess the capabilities of Mixtral to tackle long context, we evaluate it 
on the passkey retrieval task introduced in [23], a synthetic task designed to measure the ability of the model to 
retrieve a passkey inserted randomly in a long prompt. Results in Figure 4 (Left) show that Mixtral achieves a 100%
retrieval accuracy regardless of the context length or the position of passkey in the sequence. Figure 4 (Right) 
shows that the perplexity of Mixtral on a subset of the proof-pile dataset [2] decreases monotonically as the size 
of the context increases. Passkey Performance ry 0.8 0.6 04 0.2 0.0 OK 4K 8K 12K 16K 20K 24K 28K Seq Len Passkey 
Loc\n\nThe chunk discusses Mixtral's performance on multilingual benchmarks and its ability to handle long-range 
context, demonstrating its strong capabilities in these areas.", 'metadata': {'title': 'Mixtral of Experts', 
'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 2 (score: 2.11): {'text': 'expertsâ ) to process the token and combine their output additively. This technique
increases the number of parameters of a model while controlling cost and latency, as the model only uses a fraction
of the total set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k 
tokens. It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In 
particular, Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key 
architectural details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger 
models like Llama 2 70B and GPT-3.5 on various benchmarks.', 'metadata': {'title': 'Mixtral of Experts', 
'arxiv_id': '2401.04088', 'references': ['1905.07830']}}